In [2]:
import requests
from requests_html import HTMLSession

In [ ]:
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36',

}

github_url = "https://github.com"
list_title = []
list_title_url = []
list_text = []
def get_title_and_text(resp, pages_next):
    button = resp.html.xpath('//div[@class="BtnGroup"]/a[@rel="nofollow"]/text()')
#     print(button)
    if len(pages_next) == 0 or (len(pages_next) == 1 and button[0] == "Next"):
        title = resp.html.xpath('//div[@class="d-inline-block mb-1"]/h3/a/@href')
        title_url = [github_url + i for i in title]
        text = resp.html.xpath('//div[@class="py-1"]/p/text()')
        text = [j.replace("\n      ","") for j in [i.replace("\n        ","") for i in text]]
        
        list_title.append(title)
        list_title_url.append(title_url)
        list_text.append(text)
        
        return list_title,list_title_url,list_text
        
    elif len(pages_next) == 2:
        title = resp.html.xpath('//div[@class="d-inline-block mb-1"]/h3/a/@href')
        title_url = [github_url + i for i in title]
        text = resp.html.xpath('//div[@class="py-1"]/p/text()')
        text = [j.replace("\n      ","") for j in [i.replace("\n        ","") for i in text]]
        
        list_title.append(title)
        list_title_url.append(title_url)
        list_text.append(text)
        
        resp = session.get(pages_next[1], headers=headers)
        pages_next = resp.html.xpath('//div[@class="BtnGroup"]/a[@rel="nofollow"]/@href')
        get_title_and_text(resp, pages_next)
        
    

def run(username):
    stars_url = "https://github.com/" + username + "?tab=stars"
    session = HTMLSession()
    resp = session.get(stars_url, headers=headers)
    
    # 获取类型
    types = resp.html.xpath('//ul[@class="filter-list"]/li/a/text()')
    types_result = [k.replace("  ","") for k in [j.replace("\n      ","") for j in [i.replace("\n        ","") for i in types]]]
    types_result = types_result[4:] # 去掉多余
#     print(types_result)
    
    # 获取类型链接
    urls = resp.html.xpath('//ul[@class="filter-list"]/li/a/@href')
    urls_result = urls[4:] # 去掉多余
    print(urls_result)
    
    list_title = []
    list_title_url = []
    list_text = []
    # 每个类型的页面
    for url in urls_result:
        resp = session.get(url, headers=headers)
        pages_next = resp.html.xpath('//div[@class="BtnGroup"]/a[@rel="nofollow"]/@href')
#         type_data = get_title_and_text(resp, pages_next)
        
#         list_title.append(type_data[0])
#         list_title_url.append(type_data[1])
#         list_text.append(type_data[2])

#     print(list_title)
#     print(list_title_url)
#     print(list_text)
    
    # 获取项目名称、评论、stars、fork
    # 判断是否有下一页，获取链接
    
def makeMarkdown():
    details_head_contents = "<details><summary>Contents</summary>"
    details_head_List = "<details><summary>List</summary>"
    datails_font = "</details>"
    
    # 模板制作：显示样式，是否需要stars&fork
    # 支持输入文件名，可有可无".md"
    
if __name__ == '__main__':
#     username = input("input your github username: ")
    username = "anlzou"
    data = run(username)

In [4]:
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36',
}

'''
    data[0]:types
    data[1]:types_url
    list_title:project name
    list_title_url:project url
    list_text:project brief introduction
    
'''
data = []
github_url = "https://github.com"
session = HTMLSession()

list_title = []
list_title_url = []
list_text = []

def get_title_and_text(url):
    resp = session.get(url, headers=headers)
    url = resp.html.xpath('//div[@class="BtnGroup"]/a[@rel="nofollow"]/@href')
    button = resp.html.xpath('//div[@class="BtnGroup"]/a[@rel="nofollow"]/text()')

    if len(url) == 0 or (len(url) == 1 and button[0] == "Next"):
        title = resp.html.xpath('//div[@class="d-inline-block mb-1"]/h3/a/@href')
        title_url = [github_url + i for i in title]
        text = resp.html.xpath('//div[@class="py-1"]/p/text()')
        text = [j.replace("\n      ","") for j in [i.replace("\n        ","") for i in text]]
        
        list_title.append(title)
        list_title_url.append(title_url)
        list_text.append(text)
        
#         return list_title,list_title_url,list_text
        
    elif len(pages_next) == 2:
        title = resp.html.xpath('//div[@class="d-inline-block mb-1"]/h3/a/@href')
        title_url = [github_url + i for i in title]
        text = resp.html.xpath('//div[@class="py-1"]/p/text()')
        text = [j.replace("\n      ","") for j in [i.replace("\n        ","") for i in text]]
        
        list_title.append(title)
        list_title_url.append(title_url)
        list_text.append(text)
        
        resp = session.get(pages_next[1], headers=headers)
        pages_next = resp.html.xpath('//div[@class="BtnGroup"]/a[@rel="nofollow"]/@href')
        get_title_and_text(resp)
        
def get_types(username):
    stars_url = "https://github.com/" + username + "?tab=stars"
    resp = session.get(stars_url, headers=headers)
    
    # 获取类型
    types = resp.html.xpath('//ul[@class="filter-list"]/li/a/text()')
    types_result = [k.replace("  ","") for k in [j.replace("\n      ","") for j in [i.replace("\n        ","") for i in types]]]
    types_result = types_result[4:] # 去掉多余
    
    # 获取类型链接
    urls = resp.html.xpath('//ul[@class="filter-list"]/li/a/@href')
    urls_result = urls[4:] # 去掉多余

    return types_result,urls_result

def run(username):
    type_data = get_types(username)
    for i in type_data:
        data.append(i)
        
#     每个类型的页面
    for url in data[1]:
        get_title_and_text(url)
    
#     print(list_title)
#     print(list_title_url)
#     print(list_text)
        
#     for i in data:
#         print(i)
    
    # 获取项目名称、评论、stars、fork
    
def makeMarkdown():
    details_head_contents = "<details><summary>Contents</summary>"
    details_head_List = "<details><summary>List</summary>"
    datails_font = "</details>"
    
    # 模板制作：显示样式，是否需要stars&fork
    # 支持输入文件名，可有可无".md"
    
if __name__ == '__main__':
#     username = input("input your github username: ")
    username = "anlzou"
    run(username)

[['/taichi-dev/taichi'], ['/jdan/98.css'], ['/chokcoco/magicCss', '/bennettfeely/bennett'], ['/CyC2018/CS-Notes', '/doocs/advanced-java', '/kon9chunkit/GitHub-Chinese-Top-Charts', '/halo-dev/halo', '/Antabot/White-Jotter', '/enilu/web-flash', '/ProceduralZC/pointZSXY'], ['/apache/incubator-echarts', '/transloadit/uppy', '/sampotts/plyr', '/fingerprintjs/fingerprintjs2', '/jupyterlab/jupyterlab', '/cmiscm/leonsans', '/metafizzy/zdog', '/ElemeFE/v-charts', '/timqian/chinese-independent-blogs', '/plotly/falcon', '/mengshukeji/Luckysheet', '/OBKoro1/koro1FileHeader', '/coreui/coreui-icons', '/pchen66/panolens.js', '/mayeaux/videodownloader', '/ColinEspinas/darken', '/Mongkii/RMind', '/vortesnail/qier-player'], ['/jobbole/awesome-python-cn'], ['/521xueweihan/HelloGitHub', '/fxsjy/jieba', '/pyecharts/pyecharts', '/wistbean/learn_python3_spider', '/tebelorg/RPA-Python', '/maguowei/starred', '/JustDoPython/python-examples'], ['/nvm-sh/nvm'], ['/cdr/code-server', '/excalidraw/excalidraw', '/cyr